# Part One

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob
import copy 

### Pulling Info From YML File

In [2]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds41_42_config.yml'

In [3]:
config_filename

'./ds41_42_config.yml'

In [4]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
first_ID = config['inputs']['first_ID']

In [6]:
second_ID = config['inputs']['second_ID']

In [7]:
output_path = config['inputs']['output_path']

In [8]:
first_ID_datafiles = config[first_ID].values()

In [9]:
second_ID_datafiles = config[second_ID].values()

### Downloading SQL Data

In [10]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [11]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [12]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [13]:
# SQl Data at different levels
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction'].drop('geotype', axis=1)
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa'].drop('geotype', axis=1)

### Downloading the first_ID Data

In [14]:
# first_ID Data Frame
first_ID_df = pd.DataFrame()
for file_name in first_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    first_ID_df = first_ID_df.append(working_df)

In [15]:
# Save the features_first_ID for future use (Used when creating the diff file)
features_first_ID = first_ID_df.drop(['mgra', 'year'], axis=1).columns

In [16]:
comparison_first_ID_no_geozone = copy.deepcopy(first_ID_df)

In [17]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data = comparison_first_ID_no_geozone.merge(jur_level, how='left', on='mgra')

In [18]:
# making it original
comparison_first_ID_processed_data.columns = [x + f'_{first_ID}' for x in comparison_first_ID_processed_data.columns]

In [19]:
comparison_first_ID_processed_data.head()

,mgra_DS35,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,...,hotelroomtotal_DS35,luz_id_DS35,truckregiontype_DS35,district27_DS35,milestocoast_DS35,acres_DS35,effective_acres_DS35,land_acres_DS35,year_DS35,geozone_DS35
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego


In [20]:
#comparison_first_ID_processed_data.to_csv(f"{output_path}/comparison_{first_ID}_processed_data.csv")

### Downloading  the second_ID Data

In [21]:
# second_id Data Frame
second_ID_df = pd.DataFrame()
for file_name in second_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    second_ID_df = second_ID_df.append(working_df)

In [22]:
# Save the features of the second ID for future use (Used when creating the diff file)
features_second_ID = second_ID_df.drop(['mgra', 'year'], axis=1).columns

In [23]:
comparison_second_ID_no_geozone = copy.deepcopy(second_ID_df)

In [24]:
# Adding SQl Data to second_id_df
comparison_second_ID_processed_data = comparison_second_ID_no_geozone.merge(jur_level, how='left', on='mgra')

In [25]:
# making it original
comparison_second_ID_processed_data.columns = [x + f'_{second_ID}' for x in comparison_second_ID_processed_data.columns]

In [26]:
comparison_second_ID_processed_data.head()

,mgra_DS41,taz_DS41,hs_DS41,hs_sf_DS41,hs_mf_DS41,hs_mh_DS41,hh_DS41,hh_sf_DS41,hh_mf_DS41,hh_mh_DS41,...,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41,geozone_DS41
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego


In [27]:
# comparison_second_ID_processed_data.to_csv(f"{output_path}/comparison_{second_ID}_processed_data.csv")

### Concatenate both DS dataframes

In [28]:
# Inner join dataframes on mgra and year
first_second_ID_combined = first_ID_df.merge(second_ID_df, on=["mgra", "year"], suffixes=[f'_{first_ID}', f'_{second_ID}'])

In [29]:
first_second_ID_comparison = copy.deepcopy(first_second_ID_combined)

In [30]:
# Aggregate sum by mgra and year values
first_second_ID_comparison = first_second_ID_comparison.groupby(['mgra', 'year']).sum()

In [31]:
first_second_ID_comparison.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra year                                                                   
1    2016      3331       19          19           0           0       18   
     2018      3331       19          19           0           0       18   
     2020      3331       19          19           0           0       18   
     2023      3331       20          20           0           0       18   
     2025      3331       20          20           0           0       18   

           hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra year                                                   ...   
1    2016          18           0           0            0  ...   
     2018          18           0           0            0  ...   
     2020          18           0           0            0  ...   
     2023          18           0           0            0  ...   
     2025          18           0           0            0  ...   

           midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra year                                                             
1    2016                  0                 0                    0   
     2018                  0                 0                    0   
     2020                  0                 0                    0   
     2023                  0                 0                    0   
     2025                  0                 0                    0   

           luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra year                                                       
1    2016           95                     1               27   
     2018           95                     1               27   
     2020           95                     1               27   
     2023           95                     1               27   
     2025           95                     1               27   

           milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra year                                                        
1    2016             3.7997   16.615444             12.961482   
     2018             3.7997   16.615444             12.961482   
     2020             3.7997   16.615444             12.961482   
     2023             3.7997   16.615444             12.961482   
     2025             3.7997   16.615444             12.961482   

           land_acres_DS41  
mgra year                   
1    2016        16.615444  
     2018        16.615444  
     2020        16.615444  
     2023        16.615444  
     2025        16.615444  

[5 rows x 206 columns]

In [32]:
#first_second_ID_comparison.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison")

### Creating the complete Diff File

In [33]:
# Finding features common to both DSID data frames
list1_as_set = set(features_first_ID)
intersection = list1_as_set.intersection(features_second_ID)

shared_features = list(intersection)

In [34]:
# Calculate diff values between the two DS_ID's
diff_df = pd.DataFrame()
for column in shared_features:
    diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']

In [35]:
diff_df.head()

openspaceparkpreserve_diff  i1_diff  emp_personal_svcs_retail_diff  \
mgra year                                                                       
1    2016                         0.0        0                              0   
     2018                         0.0        0                              0   
     2020                         0.0       -1                              0   
     2023                         0.0       -1                              0   
     2025                         0.0       -1                              0   

           pseudomsa_diff  totint_diff  parkactive_diff  i7_diff  \
mgra year                                                          
1    2016               0            0              0.0        0   
     2018               0            0              0.0        3   
     2020               0            0              0.0        1   
     2023               0            0              0.0        1   
     2025               0            0              0.0       -2   

           hch_dist_diff  emp_const_non_bldg_office_diff  i2_diff  ...  \
mgra year                                                          ...   
1    2016              0                               0        0  ...   
     2018              0                               0       -1  ...   
     2020              0                               0       -2  ...   
     2023              0                               0        0  ...   
     2025              0                               0       -1  ...   

           i5_diff  emp_whsle_whs_diff  dstallssam_diff  empdenbin_diff  \
mgra year                                                                 
1    2016        0                   0                0               0   
     2018        0                   0                0               0   
     2020        0                   0                0               0   
     2023        0                   0                0               0   
     2025        2                   0                0               0   

           emp_fed_mil_diff  emp_const_non_bldg_prod_diff  economyroom_diff  \
mgra year                                                                     
1    2016                 0                             0                 0   
     2018                 0                             0                 0   
     2020                 0                             0                 0   
     2023                 0                             0                 0   
     2025                 0                             0                 0   

           enrollgradekto8_diff  luxuryroom_diff  emp_utilities_office_diff  
mgra year                                                                    
1    2016                     0                0                          0  
     2018                     0                0                          0  
     2020                     0                0                          0  
     2023                     0                0                          0  
     2025                     0                0                          0  

[5 rows x 103 columns]

In [36]:
#diff_df.to_csv(f"{output_path}/{first_ID}_{second_ID}_complete_diff")

## CPA level Data

In [37]:
# Adding SQl Data to first_second_ID_combined dataframe
comparison_processed_data_cpa = first_second_ID_combined.merge(cpa_level, how='left', on='mgra')

In [38]:
# Remove mgra column
comparison_processed_data_cpa = comparison_processed_data_cpa.drop('mgra', axis=1)

In [39]:
# Aggregate values by CPA and year (Included code to see "*Not in a CPA*" values)
comparison_processed_data_cpa = comparison_processed_data_cpa.groupby(['geozone', 'year']).sum().drop('*Not in a CPA*')

In [40]:
comparison_processed_data_cpa.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  \
geozone                   year                                              
32nd Street Naval Station 2016     86368      161          60         101   
                          2018     86368      163          62         101   
                          2020     86368      163          62         101   
                          2023     86368      163          62         101   
                          2025     86368      163          62         101   

                                hs_mh_DS35  hh_DS35  hh_sf_DS35  hh_mf_DS35  \
geozone                   year                                                
32nd Street Naval Station 2016           0      161          60         101   
                          2018           0      162          61         101   
                          2020           0      162          61         101   
                          2023           0      162          61         101   
                          2025           0      162          61         101   

                                hh_mh_DS35  gq_civ_DS35  ...  \
geozone                   year                           ...   
32nd Street Naval Station 2016           0            0  ...   
                          2018           0            0  ...   
                          2020           0            0  ...   
                          2023           0            0  ...   
                          2025           0            0  ...   

                                midpriceroom_DS41  upscaleroom_DS41  \
geozone                   year                                        
32nd Street Naval Station 2016                  0                 0   
                          2018                  0                 0   
                          2020                  0                 0   
                          2023                  0                 0   
                          2025                  0                 0   

                                hotelroomtotal_DS41  luz_id_DS41  \
geozone                   year                                     
32nd Street Naval Station 2016                   25         2538   
                          2018                   28         2538   
                          2020                   28         2538   
                          2023                   29         2538   
                          2025                   29         2538   

                                truckregiontype_DS41  district27_DS41  \
geozone                   year                                          
32nd Street Naval Station 2016                    20              200   
                          2018                    20              200   
                          2020                    20              200   
                          2023                    20              200   
                          2025                    20              200   

                                milestocoast_DS41  acres_DS41  \
geozone                   year                                  
32nd Street Naval Station 2016            56.2544  837.436033   
                          2018            56.2544  837.436033   
                          2020            56.2544  837.436033   
                          2023            56.2544  837.436033   
                          2025            56.2544  837.436033   

                                effective_acres_DS41  land_acres_DS41  
geozone                   year                                         
32nd Street Naval Station 2016            729.800231       730.222882  
                          2018            729.800231       730.222882  
                          2020            729.800231       730.222882  
                          2023            729.800231       730.222882  
                          2025            729.800231       730.222882  

[5 rows x 206 columns]

In [41]:
#comparison_processed_data_cpa.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_cpa")

## Jurisdiction level Data

In [42]:
# Adding SQl Data to first_second_ID_combined dataframe
comparison_processed_data_jur = first_second_ID_combined.merge(jur_level, how='left', on='mgra')

In [43]:
# Remove mgra column
comparison_processed_data_jur = comparison_processed_data_jur.drop('mgra', axis=1)

In [44]:
# Aggregate values by Jurisdiction and year (Included code to see "*Not in a CPA*" values)
comparison_processed_data_jur = comparison_processed_data_jur.groupby(['geozone', 'year']).sum()

In [45]:
comparison_processed_data_jur.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
geozone  year                                                                   
Carlsbad 2016    870633    50670       34815       14215        1640    47670   
         2018    870633    51501       35865       13994        1642    48718   
         2020    870633    52666       36426       14598        1642    49783   
         2023    870633    53363       36634       15087        1642    50383   
         2025    870633    53899       36987       15270        1642    50868   

               hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
geozone  year                                                   ...   
Carlsbad 2016       32817       13351        1502          915  ...   
         2018       33985       13229        1504          915  ...   
         2020       34559       13719        1505          915  ...   
         2023       34761       14117        1505          915  ...   
         2025       35050       14313        1505          915  ...   

               midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
geozone  year                                                             
Carlsbad 2016                940               765                 3406   
         2018               1026               837                 3722   
         2020               1026               837                 3722   
         2023               1056               862                 3833   
         2025               1076               879                 3905   

               luz_id_DS41  truckregiontype_DS41  district27_DS41  \
geozone  year                                                       
Carlsbad 2016        21520                   811              943   
         2018        21520                   811              943   
         2020        21520                   811              943   
         2023        21520                   811              943   
         2025        21520                   811              943   

               milestocoast_DS41    acres_DS41  effective_acres_DS41  \
geozone  year                                                          
Carlsbad 2016          1918.0907  27460.971113          16381.889663   
         2018          1918.0907  27460.971113          16381.889663   
         2020          1918.0907  27460.971113          16381.889663   
         2023          1918.0907  27460.971113          16381.889663   
         2025          1918.0907  27460.971113          16381.889663   

               land_acres_DS41  
geozone  year                   
Carlsbad 2016     26574.857225  
         2018     26574.857225  
         2020     26574.857225  
         2023     26574.857225  
         2025     26574.857225  

[5 rows x 206 columns]

In [46]:
#comparison_processed_data_jur.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_jur")

In [47]:
# Unsure about file name, maybe spell out jurisdiction instead of jur?

In [ ]:
# Should we rename geozone column to CPA or jurisdi

## Individual Level?

# Brainstorming for Part 2

In [48]:
config['dof']['dof_data']

'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-01 Regional Forecast DS 41-42/dof_data.xlsx'

In [49]:
first_second_ID_comparison.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra year                                                                   
1    2016      3331       19          19           0           0       18   
     2018      3331       19          19           0           0       18   
     2020      3331       19          19           0           0       18   
     2023      3331       20          20           0           0       18   
     2025      3331       20          20           0           0       18   

           hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra year                                                   ...   
1    2016          18           0           0            0  ...   
     2018          18           0           0            0  ...   
     2020          18           0           0            0  ...   
     2023          18           0           0            0  ...   
     2025          18           0           0            0  ...   

           midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra year                                                             
1    2016                  0                 0                    0   
     2018                  0                 0                    0   
     2020                  0                 0                    0   
     2023                  0                 0                    0   
     2025                  0                 0                    0   

           luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra year                                                       
1    2016           95                     1               27   
     2018           95                     1               27   
     2020           95                     1               27   
     2023           95                     1               27   
     2025           95                     1               27   

           milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra year                                                        
1    2016             3.7997   16.615444             12.961482   
     2018             3.7997   16.615444             12.961482   
     2020             3.7997   16.615444             12.961482   
     2023             3.7997   16.615444             12.961482   
     2025             3.7997   16.615444             12.961482   

           land_acres_DS41  
mgra year                   
1    2016        16.615444  
     2018        16.615444  
     2020        16.615444  
     2023        16.615444  
     2025        16.615444  

[5 rows x 206 columns]

In [50]:
# MGRA level stats
first_second_ID_comparison.describe(percentiles=[0.5])

,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,gq_civ_DS35,...,midpriceroom_DS41,upscaleroom_DS41,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41
count,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,...,299026.000000,299026.000000,299026.000000,299026.000000,299026.0,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000
mean,2320.266716,57.944169,32.752570,23.360668,1.830931,54.668711,31.188883,21.732037,1.747791,3.211012,...,0.784136,0.665678,2.885993,105.691809,1.0,9.612208,10.664608,118.563783,54.397386,117.319734
std,1425.802241,103.303556,58.620976,83.030109,20.385126,97.264616,56.679026,77.060780,19.453506,75.134082,...,14.700628,15.896093,35.864336,63.717413,0.0,7.310535,11.072095,1059.868200,688.381608,1058.038890
min,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.0,1.000000,0.010300,0.131722,0.000000,0.000000
50%,2217.000000,28.000000,15.000000,0.000000,0.000000,26.000000,14.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,105.000000,1.0,10.000000,7.491400,17.136542,14.699567,17.041284
max,4996.000000,5811.000000,4144.000000,4000.000000,802.000000,5573.000000,4115.000000,3095.000000,770.000000,5496.000000,...,931.000000,1254.000000,2136.000000,229.000000,1.0,27.000000,70.450400,99278.194490,99094.679110,99136.666380
